In [9]:
import os
from dotenv import load_dotenv
import sqlite3
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box

In [10]:
# Load environment variables
load_dotenv()

console = Console()

In [11]:
DB_FILE = os.getenv("DB_FILE", "game_database.db")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [12]:
def get_connection():
    conn = sqlite3.connect(DB_FILE, timeout=10)  # Wait up to 10 seconds
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA key = '{DB_PASSWORD}';")   # Your existing key setting
    cursor.execute("PRAGMA foreign_keys = ON;")        # Enable foreign key support
    cursor.close()
    return conn


In [13]:
def execute_query(query: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()

        console.print("[bold cyan]📡 Executing Query...[/bold cyan]")
        cursor.execute(query)

        if query.strip().lower().startswith("select"):
            rows = cursor.fetchall()
            if rows:
                columns = [description[0] for description in cursor.description]

                table = Table(title="Query Results", box=box.ROUNDED, show_lines=True)
                for col in columns:
                    table.add_column(col, style="bold magenta")

                for row in rows:
                    table.add_row(*[str(cell) for cell in row])

                console.print(table)
                console.print(f"✅ [green]{len(rows)} rows fetched successfully.[/green]")
            else:
                console.print(Panel("[yellow]⚠️ Query returned no rows.[/yellow]", title="No Data", style="yellow"))
        else:
            conn.commit()
            console.print(Panel(f"✅ [green]Query executed successfully.[/green]", title="Success", style="green"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error executing query:[/red] {str(e)}", title="Error", style="red"))

    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [14]:
def list_tables():
    """List all tables with row counts."""
    query = """
    SELECT name FROM sqlite_master 
    WHERE type='table' AND name NOT LIKE 'sqlite_%'
    ORDER BY name;
    """
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(query)
        tables = cursor.fetchall()

        if not tables:
            console.print(Panel("[yellow]⚠️ No tables found in the database.[/yellow]", title="No Tables", style="yellow"))
            return

        table = Table(title="Tables in Database", box=box.ROUNDED, show_lines=True)
        table.add_column("Table Name", style="bold green")
        table.add_column("Number of Records", style="bold cyan", justify="right")

        for (table_name,) in tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            table.add_row(f"📋 {table_name}", str(count))

        console.print(table)
    except Exception as e:
        console.print(Panel(f"❗ [red]Error listing tables:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [15]:
def table_info(table_name: str):
    """Show info about table: columns, row count, and sample rows (max 10)."""
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Check if table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            console.print(Panel(f"⚠️ Table '{table_name}' does not exist.", style="yellow"))
            return

        # Get columns info
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns_info = cursor.fetchall()
        columns = [col[1] for col in columns_info]

        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]

        # Print columns and row count
        info_panel = Panel.fit(
            f"📋 [bold green]Table:[/bold green] {table_name}\n"
            f"🔢 [bold cyan]Columns:[/bold cyan] {', '.join(columns)}\n"
            f"📊 [bold magenta]Total Rows:[/bold magenta] {row_count}",
            title="Table Info",
            style="blue",
        )
        console.print(info_panel)

        # Fetch and show up to 10 rows
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
        rows = cursor.fetchall()

        if rows:
            table = Table(title=f"Sample Data from {table_name}", box=box.ROUNDED, show_lines=True)
            for col in columns:
                table.add_column(col, style="bold magenta")
            for row in rows:
                table.add_row(*[str(cell) for cell in row])
            console.print(table)
        else:
            console.print(Panel("[yellow]⚠️ No data found in the table.[/yellow]", title="Empty Table", style="yellow"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error fetching table info:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [16]:
list_tables()

            Tables in Database            
╭────────────────────┬───────────────────╮
│ Table Name         │ Number of Records │
├────────────────────┼───────────────────┤
│ 📋 achievements    │                 2 │
├────────────────────┼───────────────────┤
│ 📋 avatars         │                 0 │
├────────────────────┼───────────────────┤
│ 📋 leagues         │                13 │
├────────────────────┼───────────────────┤
│ 📋 legs            │                 4 │
├────────────────────┼───────────────────┤
│ 📋 matches         │                 4 │
├────────────────────┼───────────────────┤
│ 📋 matches_new     │                 0 │
├────────────────────┼───────────────────┤
│ 📋 players         │                 2 │
├────────────────────┼───────────────────┤
│ 📋 predictions     │                19 │
├────────────────────┼───────────────────┤
│ 📋 rounds          │                 3 │
├────────────────────┼───────────────────┤
│ 📋 stages          │                33 │
├────────────────────┼───────────────────┤
│ 📋 team_league     │               113 │
├────────────────────┼───────────────────┤
│ 📋 teams           │                57 │
├────────────────────┼───────────────────┤
│ 📋 two_legged_ties │                 2 │
╰────────────────────┴───────────────────╯

🔌 Connection closed.

In [20]:
def drop_tables_interactive():
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Get list of user tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            print("No user tables found to drop.")
            return

        print("\n📋 Existing tables:")
        for idx, table in enumerate(tables, 1):
            print(f"{idx}. {table}")

        print("\nOptions:")
        print("  🔢 Enter the table number to drop a specific table")
        print("  💣 Enter 'all' to drop ALL tables")
        print("  ❌ Enter 'exit' to cancel and exit")

        while tables:
            choice = input("\nYour choice: ").strip().lower()

            if choice == 'exit':
                print("🚪 Exit without dropping tables.")
                break

            elif choice == 'all':
                for table in tables:
                    try:
                        cursor.execute(f"DROP TABLE IF EXISTS {table};")
                        print(f"✅ Dropped table '{table}'")
                    except sqlite3.OperationalError as e:
                        print(f"❌ Failed to drop '{table}': {e}")
                conn.commit()
                break

            elif choice.isdigit():
                idx = int(choice)
                if 1 <= idx <= len(tables):
                    table = tables[idx - 1]
                    try:
                        cursor.execute(f"DROP TABLE IF EXISTS {table};")
                        conn.commit()
                        print(f"✅ Dropped table '{table}'")
                        tables.pop(idx - 1)
                    except sqlite3.OperationalError as e:
                        print(f"❌ Failed to drop '{table}': {e}")

                    if not tables:
                        print("📭 No more tables left.")
                        break
                    else:
                        print("\n📋 Remaining tables:")
                        for i, t in enumerate(tables, 1):
                            print(f"{i}. {t}")
                else:
                    print("⚠️ Invalid table number, try again.")
            else:
                print("⚠️ Invalid input, try again.")

    except sqlite3.OperationalError as e:
        print(f"🚫 Database operation failed: {e}")
    finally:
        try:
            cursor.close()
        except:
            pass
        try:
            conn.close()
        except:
            pass


In [13]:
drop_tables_interactive()


📋 Existing tables:
1. rounds
2. teams
3. matches
4. legs
5. avatars
6. predictions
7. achievements
8. team_league
9. players
10. leagues

Options:
  🔢 Enter the table number to drop a specific table
  💣 Enter 'all' to drop ALL tables
  ❌ Enter 'exit' to cancel and exit
❌ Failed to drop 'teams': database is locked

📋 Remaining tables:
1. rounds
2. teams
3. matches
4. legs
5. avatars
6. predictions
7. achievements
8. team_league
9. players
10. leagues
🚪 Exit without dropping tables.


## 1.1 🧑‍💼 1. Players Table

In [ ]:
query_players = """
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    avatar_url TEXT,
    role TEXT CHECK(role IN ('admin', 'player')) DEFAULT 'player',
    is_confirmed INTEGER DEFAULT 1,
    reset_token TEXT,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
    last_login_at TEXT DEFAULT NULL
);
"""
execute_query(query_players)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 2.2 🏆 2. Leagues Table

In [ ]:
query_leagues = """
CREATE TABLE IF NOT EXISTS leagues (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE NOT NULL,
    country TEXT,
    logo_path TEXT,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_leagues)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 3.3 📅 3. Rounds Table

In [12]:
query_rounds = """
CREATE TABLE IF NOT EXISTS rounds (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    start_date TEXT NOT NULL,
    end_date TEXT NOT NULL,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_rounds)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 4.4 🏟️ 4. Teams Table

In [ ]:
query_teams = """
CREATE TABLE IF NOT EXISTS teams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    logo_path TEXT,
    league_id INTEGER NOT NULL,
    nationality TEXT,  -- New column added here
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE
);
"""

execute_query(query_teams)


📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: database is locked                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: table teams already exists                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [54]:
alter_query = "ALTER TABLE teams ADD COLUMN Official_name Text NOT NULL;"
execute_query(alter_query)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## team_league join table

In [ ]:
execute_query("""
CREATE TABLE IF NOT EXISTS team_league (
    team_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    PRIMARY KEY (team_id, league_id),
    FOREIGN KEY (team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE
);
""")



📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 5.5 ⚽ 5. Matches Table

In [ ]:
query_matches = """
CREATE TABLE IF NOT EXISTS matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    round_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    home_team_id INTEGER NOT NULL,
    away_team_id INTEGER NOT NULL,
    match_datetime TEXT NOT NULL,
    status TEXT CHECK (status IN ('upcoming', 'live', 'finished', 'cancelled')) DEFAULT 'upcoming',
    home_score INTEGER DEFAULT NULL,
    away_score INTEGER DEFAULT NULL,
    stage_id INTEGER,  -- Allow NULL
    penalty_winner INTEGER DEFAULT NULL,  -- New column (can be home_team_id or away_team_id)
    FOREIGN KEY (round_id) REFERENCES rounds(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    FOREIGN KEY (home_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (away_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (stage_id) REFERENCES stages(id) ON DELETE CASCADE
);
"""
execute_query(query_matches)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [127]:
query_copy_data_nullable = """
ALTER TABLE matches ADD COLUMN penalty_winner INTEGER DEFAULT NULL;
"""

execute_query(query_copy_data_nullable)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [40]:
query_drop_old_matches = "DROP TABLE matches;"
execute_query(query_drop_old_matches)
query_rename_table = "ALTER TABLE matches_new RENAME TO matches;"
execute_query(query_rename_table)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 6.6 🔁 6. Stages Table

In [ ]:
query_stages = """
CREATE TABLE IF NOT EXISTS stages (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    league_id INTEGER NOT NULL,
    name TEXT NOT NULL,  -- e.g., 'Group Stage', 'Round of 16', etc.
    stage_order INTEGER NOT NULL,  -- e.g., 1 for group, 2 for round of 16, etc.
    can_be_draw INTEGER DEFAULT 1,
    two_legs INTEGER DEFAULT 0,
    must_have_winner INTEGER DEFAULT 0,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    UNIQUE (league_id, name)  -- avoid duplicate stage names per league
);
"""
execute_query(query_stages)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [38]:
def get_leagues_and_choose(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM leagues")
    leagues = cursor.fetchall()

    if not leagues:
        print("⚠️ No leagues found. Please insert a league first.")
        return None

    print("Available leagues:")
    for league in leagues:
        print(f"{league[0]}. {league[1]}")

    try:
        league_id = int(input("👉 Enter the ID of the league to assign the temporary stage to: "))
        if any(league[0] == league_id for league in leagues):
            return league_id
        else:
            print("❌ Invalid league ID.")
            return None
    except ValueError:
        print("❌ Please enter a valid number.")
        return None
    
def insert_temp_stage(conn, league_id):
    cursor = conn.cursor()
    stage_name = "Temp Stage"
    try:
        cursor.execute("""
            INSERT INTO stages (league_id, name, stage_order)
            VALUES (?, ?, ?)
        """, (league_id, stage_name, 1))
        conn.commit()
        print(f"✅ Temporary stage '{stage_name}' added with ID {cursor.lastrowid}")
        return cursor.lastrowid
    except Exception as e:
        print(f"❌ Error inserting temporary stage: {e}")
        return None

conn = get_connection()
# Assuming you're using SQLite and have a connection `conn`
league_id = get_leagues_and_choose(conn)
if league_id:
    stage_id = insert_temp_stage(conn, league_id)
    if stage_id:
        # Now use this stage_id in your matches_new insert
        print(f"✅ Use stage_id = {stage_id} when copying your matches data.")



Available leagues:
17. Bundesliga
16. CAF Champions League
7. Club World Championship
5. Copa del Rey
15. Egyptian Premier League
3. LaLiga
2. Premier League
4. Seria A
9. Supercopa de España
6. UEFA Champions League
19. UEFA Conference League
18. UEFA Europa League
13. UEFA Nations League
✅ Temporary stage 'Temp Stage' added with ID 1
✅ Use stage_id = 1 when copying your matches data.


In [36]:
query_legs = """
CREATE TABLE IF NOT EXISTS legs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER NOT NULL,
    leg_number INTEGER NOT NULL,
    leg_date TEXT NOT NULL,
    home_score INTEGER,
    away_score INTEGER,
    can_draw BOOLEAN DEFAULT 1,
    winner_team_id INTEGER,
    notes TEXT,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (winner_team_id) REFERENCES teams(id) ON DELETE SET NULL,
    UNIQUE (match_id, leg_number)
);
"""
execute_query(query_legs)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [ ]:
query_legs = """
CREATE TABLE IF NOT EXISTS two_legged_ties (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_leg_match_id INTEGER NOT NULL,
    second_leg_match_id INTEGER NOT NULL,
    winner_team_id INTEGER,  -- Nullable until calculated
    FOREIGN KEY (first_leg_match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (second_leg_match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (winner_team_id) REFERENCES teams(id) ON DELETE SET NULL,
    UNIQUE (first_leg_match_id, second_leg_match_id)
);
"""
execute_query(query_legs)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 7.7 🖼️ 7. Avatars Table

In [17]:
query_avatars = """
CREATE TABLE IF NOT EXISTS avatars (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_path TEXT NOT NULL,                          -- Local path to the image
    uploaded_by_player_id INTEGER,                    -- Who uploaded the avatar
    uploaded_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (uploaded_by_player_id) REFERENCES players(id) ON DELETE SET NULL
);
"""
execute_query(query_avatars)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 8.8 predictions Table

In [ ]:
query_predictions = """
CREATE TABLE predictions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    match_id INTEGER NOT NULL,
    
    predicted_home_score INTEGER NOT NULL,
    predicted_away_score INTEGER NOT NULL,
    predicted_penalty_winner TEXT,                -- Team name or NULL
    
    score INTEGER DEFAULT 0,                      -- Computed after match ends

    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    
    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    
    UNIQUE (player_id, match_id)                  -- Player can only predict once per match
);
"""
execute_query(query_delte)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 9.9 achievements Table

In [19]:
query_achievements = """
CREATE TABLE IF NOT EXISTS achievements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    
    total_leagues_won INTEGER DEFAULT 0,
    total_cups_won INTEGER DEFAULT 0,

    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    UNIQUE (player_id)
);
"""
execute_query(query_achievements)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [73]:
list_tables()

            Tables in Database            
╭────────────────────┬───────────────────╮
│ Table Name         │ Number of Records │
├────────────────────┼───────────────────┤
│ 📋 achievements    │                 0 │
├────────────────────┼───────────────────┤
│ 📋 avatars         │                 0 │
├────────────────────┼───────────────────┤
│ 📋 leagues         │                13 │
├────────────────────┼───────────────────┤
│ 📋 legs            │                 4 │
├────────────────────┼───────────────────┤
│ 📋 matches         │                10 │
├────────────────────┼───────────────────┤
│ 📋 players         │                 2 │
├────────────────────┼───────────────────┤
│ 📋 predictions     │                 7 │
├────────────────────┼───────────────────┤
│ 📋 rounds          │                 0 │
├────────────────────┼───────────────────┤
│ 📋 stages          │               100 │
├────────────────────┼───────────────────┤
│ 📋 team_league     │               113 │
├────────────────────┼───────────────────┤
│ 📋 teams           │                57 │
├────────────────────┼───────────────────┤
│ 📋 two_legged_ties │                 0 │
╰────────────────────┴───────────────────╯

🔌 Connection closed.

In [101]:
table_info("two_legged_ties")

╭────────────────────────────── Table Info ───────────────────────────────╮
│ 📋 Table: two_legged_ties                                               │
│ 🔢 Columns: id, first_leg_match_id, second_leg_match_id, winner_team_id │
│ 📊 Total Rows: 3                                                        │
╰─────────────────────────────────────────────────────────────────────────╯

                 Sample Data from two_legged_ties                 
╭────┬────────────────────┬─────────────────────┬────────────────╮
│ id │ first_leg_match_id │ second_leg_match_id │ winner_team_id │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 1  │ 42                 │ 43                  │ None           │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 2  │ 45                 │ 46                  │ None           │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 3  │ 47                 │ 48                  │ None           │
╰────┴────────────────────┴─────────────────────┴────────────────╯

🔌 Connection closed.

In [131]:
table_info("matches")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score,            │
│ away_score, stage_id, penalty_winner                                                                            │
│ 📊 Total Rows: 6                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from matches                                              
╭────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬─────────╮
│ id │ round_id │ league_… │ home_te… │ away_te… │ match_d… │ status   │ home_sc… │ away_sc… │ stage_id │ penalt… │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 57 │ 25       │ 13       │ 118      │ 121      │ 2025-06… │ finished │ 1        │ 2        │ 222      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 58 │ 25       │ 13       │ 119      │ 120      │ 2025-06… │ finished │ 4        │ 5        │ 222      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 59 │ 26       │ 13       │ 118      │ 120      │ 2025-06… │ finished │ 0        │ 2        │ 223      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 60 │ 26       │ 13       │ 121      │ 119      │ 2025-06… │ finished │ 2        │ 2        │ 224      │ 119     │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 71 │ 26       │ 6        │ 65       │ 83       │ 2025-06… │ upcoming │ None     │ None     │ 220      │ None    │
│    │          │          │          │          │ 22:00:00 │          │          │          │          │         │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 72 │ 26       │ 6        │ 83       │ 65       │ 2025-06… │ upcoming │ None     │ None     │ 220      │ None    │
│    │          │          │          │          │ 22:00:00 │          │          │          │          │         │
╰────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴─────────╯

🔌 Connection closed.

In [90]:
import sqlite3

conn = sqlite3.connect("game_database.db")
cur = conn.cursor()

try:
    cur.execute("ALTER TABLE players ADD COLUMN avatar_url TEXT")
except sqlite3.OperationalError as e:
    print("Column probably already exists:", e)

conn.commit()
conn.close()


In [76]:
def show_and_clear_teams():
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch all records from teams
    cursor.execute("SELECT * FROM matches;")
    records = cursor.fetchall()
    print("Records before deletion:")
    if records:
        for row in records:
            print(row)
    else:
        print("No records found.")

    # Delete all records from teams
    cursor.execute("DELETE FROM matches;")
    conn.commit()

    # Fetch again to confirm deletion
    cursor.execute("SELECT * FROM matches;")
    records_after = cursor.fetchall()
    print("\nRecords after deletion:")
    if records_after:
        for row in records_after:
            print(row)
    else:
        print("No records found.")

    cursor.close()
    conn.close()


In [77]:
show_and_clear_teams()

Records before deletion:
(21, 7, 13, 118, 121, '2025-06-04T22:15:00', 'finished', 1, 2, 182)
(22, 7, 13, 119, 120, '2025-06-05T22:00:00', 'finished', 5, 4, 182)
(23, 8, 13, 118, 120, '2025-06-08T16:00:00', 'finished', 0, 2, 183)
(24, 8, 13, 121, 119, '2025-06-08T22:00:00', 'finished', 2, 2, 184)
(25, 8, 3, 65, 67, '2025-06-09T22:16:00', 'finished', 1, 1, 137)
(26, 11, 6, 65, 83, '2025-04-30 22:00:00', 'finished', None, None, 196)
(27, 12, 6, 83, 65, '2025-05-06 22:00:00', 'finished', None, None, 196)
(28, 7, 6, 117, 83, '2025-05-31 22:00:00', 'finished', None, None, 198)
(29, 7, 6, 117, 71, '2025-05-31 22:00:00', 'finished', None, None, 198)
(30, 13, 6, 117, 71, '2025-05-31 22:00:00', 'upcoming', None, None, 198)


OperationalError: no such table: main.leagues_old

In [45]:
def show_and_clear_team_league():
    conn = get_connection()
    cursor = conn.cursor()
    
    print("📋 Current records in 'team_league':")
    cursor.execute("SELECT * FROM team_league;")
    rows = cursor.fetchall()
    if rows:
        for row in rows:
            print(row)
    else:
        print("No records found.")
    
    # Delete all records
    cursor.execute("DELETE FROM team_league;")
    conn.commit()
    print("\n🗑️ Deleted all records from 'team_league'.")
    
    # Display again to confirm
    cursor.execute("SELECT * FROM team_league;")
    rows_after_delete = cursor.fetchall()
    print("\n📋 Records in 'team_league' after deletion:")
    if rows_after_delete:
        for row in rows_after_delete:
            print(row)
    else:
        print("No records found.")
    
    cursor.close()
    conn.close()

# Usage:
show_and_clear_team_league()

📋 Current records in 'team_league':
(3, 11)
(5, 3)
(5, 5)
(5, 6)
(5, 7)
(5, 9)
(6, 3)
(6, 5)
(6, 6)
(6, 9)

🗑️ Deleted all records from 'team_league'.

📋 Records in 'team_league' after deletion:
No records found.


In [56]:
def show_and_clear_tables():
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    if not tables:
        print("No tables found in the database.")
        cursor.close()
        conn.close()
        return

    print("Tables in the database:")
    for i, table in enumerate(tables, 1):
        print(f"{i}. {table}")

    # Ask user to select a table
    while True:
        try:
            choice = int(input("\nEnter the number of the table you want to inspect: "))
            if 1 <= choice <= len(tables):
                selected_table = tables[choice - 1]
                break
            else:
                print(f"Please enter a number between 1 and {len(tables)}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    # Show number of records in the selected table
    cursor.execute(f"SELECT COUNT(*) FROM {selected_table};")
    record_count = cursor.fetchone()[0]
    print(f"\nTable '{selected_table}' has {record_count} record(s).")

    # Ask if user wants to delete all records
    delete_confirm = input("Do you want to delete all records in this table? (yes/no): ").strip().lower()
    if delete_confirm == "yes":
        cursor.execute(f"DELETE FROM {selected_table};")
        conn.commit()
        print(f"All records in '{selected_table}' have been deleted.")

        # Confirm deletion
        cursor.execute(f"SELECT COUNT(*) FROM {selected_table};")
        new_count = cursor.fetchone()[0]
        print(f"Records remaining in '{selected_table}': {new_count}")
    else:
        print("No records were deleted.")

    cursor.close()
    conn.close()


In [105]:
show_and_clear_tables()

Tables in the database:
1. sqlite_sequence
2. rounds
3. teams
4. legs
5. avatars
6. predictions
7. achievements
8. team_league
9. players
10. leagues
11. matches

Table 'rounds' has 2 record(s).
All records in 'rounds' have been deleted.
Records remaining in 'rounds': 0


In [102]:
table_info("matches")
table_info("rounds")
table_info("teams")
table_info("team_league")
table_info("legs")


╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score,            │
│ away_score, stage_id                                                                                            │
│ 📊 Total Rows: 10                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from matches                                              
╭────┬──────────┬───────────┬────────────┬────────────┬────────────┬──────────┬────────────┬───────────┬──────────╮
│ id │ round_id │ league_id │ home_team… │ away_team… │ match_dat… │ status   │ home_score │ away_sco… │ stage_id │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 41 │ 17       │ 6         │ FC         │ Inter      │ 2025-04-30 │ upcoming │ None       │ None      │ 196      │
│    │          │           │ Barcelona  │ Milan      │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 42 │ 17       │ 6         │ FC         │ Inter      │ 2025-04-30 │ upcoming │ None       │ None      │ 196      │
│    │          │           │ Barcelona  │ Milan      │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 43 │ 17       │ 6         │ Inter      │ FC         │ 2025-05-06 │ upcoming │ None       │ None      │ 196      │
│    │          │           │ Milan      │ Barcelona  │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 45 │ 22       │ 6         │ FC         │ Inter      │ 2025-04-30 │ upcoming │ None       │ None      │ 196      │
│    │          │           │ Barcelona  │ Milan      │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 46 │ 23       │ 6         │ Inter      │ FC         │ 2025-05-06 │ upcoming │ None       │ None      │ 196      │
│    │          │           │ Milan      │ Barcelona  │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 47 │ 22       │ 6         │ Arsenal FC │ PSG        │ 2025-04-29 │ upcoming │ None       │ None      │ 196      │
│    │          │           │            │            │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 48 │ 23       │ 6         │ PSG        │ Arsenal FC │ 2025-05-07 │ upcoming │ None       │ None      │ 196      │
│    │          │           │            │            │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 49 │ 17       │ 6         │ 117        │ 83         │ 2025-05-31 │ finished │ None       │ None      │ 198      │
│    │          │           │            │            │ 22:00:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 52 │ 24       │ 6         │ 65         │ 82         │ 2025-06-10 │ live     │ None       │ None      │ 196      │
│    │          │           │            │            │ 02:34:00   │          │            │           │          │
├────┼──────────┼───────────┼────────────┼────────────┼────────────┼──────────┼────────────┼───────────┼──────────┤
│ 53 │ 24       │ 6         │ 82         │ 65         │ 2025-06-11 │ upcoming │ None       │ None      │ 196      │
│    │          │           │            │            │ 02:34:00   │          │            │           │          │
╰────┴──────────┴───────────┴────────────┴────────────┴────────────┴──────────┴────────────┴───────────┴──────────╯

🔌 Connection closed.

╭────────────────────── Table Info ──────────────────────╮
│ 📋 Table: rounds                                       │
│ 🔢 Columns: id, name, start_date, end_date, created_at │
│ 📊 Total Rows: 2                                       │
╰────────────────────────────────────────────────────────╯

                    Sample Data from rounds                     
╭────┬─────────┬────────────┬────────────┬─────────────────────╮
│ id │ name    │ start_date │ end_date   │ created_at          │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 17 │ Round 1 │ 2025-05-31 │ 2025-06-06 │ 2025-06-09 22:01:22 │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 24 │ Round 2 │ 2025-06-07 │ 2025-06-13 │ 2025-06-09 23:35:07 │
╰────┴─────────┴────────────┴────────────┴─────────────────────╯

...

   Sample Data from    
      team_league      
╭─────────┬───────────╮
│ team_id │ league_id │
├─────────┼───────────┤
│ 65      │ 3         │
├─────────┼───────────┤
│ 65      │ 5         │
├─────────┼───────────┤
│ 65      │ 6         │
├─────────┼───────────┤
│ 65      │ 9         │
├─────────┼───────────┤
│ 66      │ 3         │
├─────────┼───────────┤
│ 66      │ 5         │
├─────────┼───────────┤
│ 66      │ 6         │
├─────────┼───────────┤
│ 67      │ 3         │
├─────────┼───────────┤
│ 67      │ 5         │
├─────────┼───────────┤
│ 67      │ 6         │
╰─────────┴───────────╯

🔌 Connection closed.

╭────────────────────────────────────────────── Table Info ───────────────────────────────────────────────╮
│ 📋 Table: legs                                                                                          │
│ 🔢 Columns: id, match_id, leg_number, leg_date, home_score, away_score, can_draw, winner_team_id, notes │
│ 📊 Total Rows: 4                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                               Sample Data from legs                                               
╭────┬──────────┬────────────┬───────────────┬────────────┬────────────┬──────────┬───────────────┬───────────────╮
│ id │ match_id │ leg_number │ leg_date      │ home_score │ away_score │ can_draw │ winner_team_… │ notes         │
├────┼──────────┼────────────┼───────────────┼────────────┼────────────┼──────────┼───────────────┼───────────────┤
│ 42 │ 21       │ 1          │ 2025-06-04T2… │ 1          │ 2          │ 1        │ 121           │ updated from  │
│    │          │            │               │            │            │          │               │ match edit    │
├────┼──────────┼────────────┼───────────────┼────────────┼────────────┼──────────┼───────────────┼───────────────┤
│ 43 │ 22       │ 1          │ 2025-06-05T2… │ 5          │ 4          │ 1        │ 119           │ updated from  │
│    │          │            │               │            │            │          │               │ match edit    │
├────┼──────────┼────────────┼───────────────┼────────────┼────────────┼──────────┼───────────────┼───────────────┤
│ 44 │ 23       │ 1          │ 2025-06-08T1… │ 0          │ 2          │ 1        │ 120           │ updated from  │
│    │          │            │               │            │            │          │               │ match edit    │
├────┼──────────┼────────────┼───────────────┼────────────┼────────────┼──────────┼───────────────┼───────────────┤
│ 45 │ 24       │ 1          │ 2025-06-08T2… │ 2          │ 2          │ 1        │ 121           │ updated from  │
│    │          │            │               │            │            │          │               │ match edit    │
╰────┴──────────┴────────────┴───────────────┴────────────┴────────────┴──────────┴───────────────┴───────────────╯

🔌 Connection closed.

In [53]:
table_info("predictions")
table_info("matches")
table_info("legs")
table_info("teams")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: predictions                                                                                           │
│ 🔢 Columns: id, player_id, match_id, predicted_home_score, predicted_away_score, predicted_penalty_winner,      │
│ score, created_at                                                                                               │
│ 📊 Total Rows: 0                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score,            │
│ away_score, stage_id                                                                                            │
│ 📊 Total Rows: 0                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

╭──────────────────────────── Table Info ─────────────────────────────╮
│ 📋 Table: teams                                                     │
│ 🔢 Columns: id, name, logo_path, league_id, created_at, nationality │
│ 📊 Total Rows: 0                                                    │
╰─────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [101]:
conn = get_connection()
result = conn.execute("PRAGMA foreign_keys").fetchone()
print("Foreign keys enabled?", result[0])
conn.close()

Foreign keys enabled? 1


In [22]:
table_info("predictions")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: predictions                                                                                           │
│ 🔢 Columns: id, player_id, match_id, predicted_home_score, predicted_away_score, predicted_penalty_winner,      │
│ score, created_at                                                                                               │
│ 📊 Total Rows: 6                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                           Sample Data from predictions                                            
╭────┬───────────┬──────────┬───────────────────┬──────────────────┬───────────────────┬───────┬──────────────────╮
│ id │ player_id │ match_id │ predicted_home_s… │ predicted_away_… │ predicted_penalt… │ score │ created_at       │
├────┼───────────┼──────────┼───────────────────┼──────────────────┼───────────────────┼───────┼──────────────────┤
│ 1  │ 8         │ 73       │ 3                 │ 1                │                   │ 0     │ 2025-06-10       │
│    │           │          │                   │                  │                   │       │ 21:32:35         │
├────┼───────────┼──────────┼───────────────────┼──────────────────┼───────────────────┼───────┼──────────────────┤
│ 2  │ 8         │ 74       │ 3                 │ 1                │                   │ 0     │ 2025-06-10       │
│    │           │          │                   │                  │                   │       │ 21:32:40         │
├────┼───────────┼──────────┼───────────────────┼──────────────────┼───────────────────┼───────┼──────────────────┤
│ 3  │ 8         │ 75       │ 4                 │ 1                │                   │ 0     │ 2025-06-10       │
│    │           │          │                   │                  │                   │       │ 21:32:45         │
├────┼───────────┼──────────┼───────────────────┼──────────────────┼───────────────────┼───────┼──────────────────┤
│ 4  │ 8         │ 76       │ 1                 │ 2                │                   │ 0     │ 2025-06-10       │
│    │           │          │                   │                  │                   │       │ 21:32:53         │
├────┼───────────┼──────────┼───────────────────┼──────────────────┼───────────────────┼───────┼──────────────────┤
│ 5  │ 8         │ 83       │ 1                 │ 3                │                   │ 0     │ 2025-06-10       │
│    │           │          │                   │                  │                   │       │ 21:45:05         │
├────┼───────────┼──────────┼───────────────────┼──────────────────┼───────────────────┼───────┼──────────────────┤
│ 6  │ 8         │ 84       │ 2                 │ 1                │                   │ 0     │ 2025-06-10       │
│    │           │          │                   │                  │                   │       │ 21:45:20         │
╰────┴───────────┴──────────┴───────────────────┴──────────────────┴───────────────────┴───────┴──────────────────╯

🔌 Connection closed.

In [117]:

table_info("predictions")
table_info("stages")
table_info("leagues")
table_info("teams")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: predictions                                                                                           │
│ 🔢 Columns: id, player_id, match_id, predicted_home_score, predicted_away_score, predicted_penalty_winner,      │
│ score, created_at                                                                                               │
│ 📊 Total Rows: 0                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

╭─────────────────────────────────────────── Table Info ────────────────────────────────────────────╮
│ 📋 Table: stages                                                                                  │
│ 🔢 Columns: id, league_id, name, stage_order, can_be_draw, two_legs, must_have_winner, created_at │
│ 📊 Total Rows: 23                                                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────────────╯

                                              Sample Data from stages                                              
╭─────┬───────────┬──────────────────┬─────────────┬─────────────┬──────────┬──────────────────┬──────────────────╮
│ id  │ league_id │ name             │ stage_order │ can_be_draw │ two_legs │ must_have_winner │ created_at       │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 199 │ 2         │ Football,        │ 1           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:24         │
│     │           │ League, Round 1  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 200 │ 2         │ Football,        │ 2           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:29         │
│     │           │ League, Round 2  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 201 │ 2         │ Football,        │ 3           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:32         │
│     │           │ League, Round 3  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 202 │ 2         │ Football,        │ 4           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:35         │
│     │           │ League, Round 4  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 203 │ 2         │ Football,        │ 5           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:39         │
│     │           │ League, Round 5  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 204 │ 2         │ Football,        │ 6           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:42         │
│     │           │ League, Round 6  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 205 │ 2         │ Football,        │ 7           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:46         │
│     │           │ League, Round 7  │             │             │          │                  │                  │
├─────┼───────────┼──────────────────┼─────────────┼─────────────┼──────────┼──────────────────┼──────────────────┤
│ 206 │ 2         │ Football,        │ 8           │ 1           │ 0        │ 0                │ 2025-06-09       │
│     │           │ England, Premier │             │             │          │                  │ 23:46:52         │
│     │           │ League, Round 8  │             │    

...

                                            Sample Data from leagues                                            
╭────┬─────────────────────────┬─────────┬───────────────────────────────────────────────┬─────────────────────╮
│ id │ name                    │ country │ logo_path                                     │ created_at          │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 2  │ Premier League          │ England │ assets/leagues\PremierLeague.jpeg             │ 2025-06-06 02:11:32 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 3  │ LaLiga                  │ Spain   │ static/uploads\LaLiga.jpeg                    │ 2025-06-06 02:12:31 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 4  │ Seria A                 │ Italy   │ static/uploads\SERIA A.png                    │ 2025-06-06 02:12:57 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 5  │ Copa del Rey            │ Spain   │ static/uploads\Copa_Del_Rey_Official_Logo.png │ 2025-06-06 02:15:00 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 6  │ UEFA Champions League   │ Europe  │ static/uploads\UEFA CHAMPIONS LEAGUE.png      │ 2025-06-06 02:16:04 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 7  │ Club World Championship │ World   │ assets/leagues\WorldChamptionClub.jpg         │ 2025-06-06 02:16:58 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 9  │ Supercopa de España     │ Spain   │ logos\league_9.png                            │ 2025-06-06 02:56:08 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 13 │ UEFA Nations League     │ Europe  │ assets/leagues\UefaNationsLeage.jpeg          │ 2025-06-06 12:01:09 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 15 │ Egyptian Premier League │ Egypt   │ assets\leagues\EgyptPremierLeague.jpeg        │ 2025-06-06 13:34:31 │
├────┼─────────────────────────┼─────────┼───────────────────────────────────────────────┼─────────────────────┤
│ 16 │ CAF Champions League    │ Africa  │ assets/leagues\CAF_Champions_League.png       │ 2025-06-06 13:47:47 │
╰────┴─────────────────────────┴─────────┴───────────────────────────────────────────────┴─────────────────────╯

🔌 Connection closed.

╭──────────────────────────────────── Table Info ────────────────────────────────────╮
│ 📋 Table: teams                                                                    │
│ 🔢 Columns: id, name, logo_path, league_id, created_at, nationality, Official_name │
│ 📊 Total Rows: 57                                                                  │
╰────────────────────────────────────────────────────────────────────────────────────╯

                                              Sample Data from teams                                               
╭────┬────────────────┬─────────────────────┬───────────┬─────────────────────┬─────────────┬─────────────────────╮
│ id │ name           │ logo_path           │ league_id │ created_at          │ nationality │ Official_name       │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 65 │ FC Barcelona   │ assets/teams/Barce… │ 3         │ 2025-06-09 16:45:08 │ Spain       │ FC Barcelona        │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 66 │ Athletic Club  │ assets/teams/Athle… │ 3         │ 2025-06-09 16:51:11 │ Spain       │ Athletic Club       │
│    │                │ Club.png            │           │                     │             │                     │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 67 │ Atl. Madrid    │ assets/teams/atlet… │ 3         │ 2025-06-09 16:52:30 │ Spain       │ Club Atlético de    │
│    │                │                     │           │                     │             │ Madrid              │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 68 │ Osasona        │ assets/teams/CA     │ 3         │ 2025-06-09 16:54:03 │ Spain       │ CA Osasuna          │
│    │                │ Osasuna.png         │           │                     │             │                     │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 69 │ Espanyol       │ assets/teams/RCD    │ 3         │ 2025-06-09 16:54:39 │ Spain       │ RCD Espanyol de     │
│    │                │ Espanyol.png        │           │                     │             │ Barcelona           │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 70 │ Getafe         │ assets/teams/Getafe │ 3         │ 2025-06-09 16:55:26 │ Spain       │ Getafe CF           │
│    │                │ CF.png              │           │                     │             │                     │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 71 │ Real Madrid    │ assets/teams/86.png │ 3         │ 2025-06-09 16:56:05 │ Spain       │ Real Madrid CF      │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 72 │ Rayo Vallecano │ assets/teams/RayoV… │ 3         │ 2025-06-09 16:57:14 │ Spain       │ Rayo Vallecano de   │
│    │                │                     │           │                     │             │ Madrid              │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 73 │ RCD Mallorca   │ assets/teams/RCD    │ 3         │ 2025-06-09 16:58:02 │ Spain       │ RCD Mallorca        │
│    │                │ Mallorca.png        │           │                     │             │                     │
├────┼────────────────┼─────────────────────┼───────────┼─────────────────────┼─────────────┼─────────────────────┤
│ 74 │ Real Betis     │ assets/teams/RealB… │ 3         │ 2025-06-09 16:58:43 │ Spain       │ Real Betis Balompié │
╰────┴────────────────┴─────────────────────┴───────────┴─────────────────────┴─────────────┴─────────────────────╯

🔌 Connection closed.

# Real Data

In [57]:
import os
import requests

# Constants
API_TOKEN = '4c936cda044a4c2eaa3ed158b90e66d3'  # Replace with your actual API key
BASE_URL = 'https://api.football-data.org/v4'
HEADERS = {'X-Auth-Token': API_TOKEN}
LOGO_DOWNLOAD_PATH = r'D:\Football_Prediction_Game\assets\clubs\Team'

# Ensure the logo directory exists
os.makedirs(LOGO_DOWNLOAD_PATH, exist_ok=True)

def get_leagues():
    """Fetches a list of top-tier football leagues."""
    url = f'{BASE_URL}/competitions'
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # Only return popular leagues
        popular_league_codes = ['PL', 'PD', 'BL1', 'SA', 'FL1', 'DED', 'PPL']
        leagues = [comp for comp in data['competitions'] if comp['code'] in popular_league_codes]
        return leagues
    else:
        print(f"Error fetching leagues: {response.status_code} - {response.text}")
        return []


def get_teams(league_id):
    """Fetches teams for a given league ID."""
    url = f'{BASE_URL}/competitions/{league_id}/teams?season=2024'
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        return data['teams']
    else:
        print(f"Error fetching teams: {response.status_code}")
        return []

def download_logo(team_name, logo_url):
    """Downloads the team's logo to the specified directory."""
    try:
        response = requests.get(logo_url, stream=True)
        if response.status_code == 200:
            # Sanitize team name for filename
            filename = f"{team_name.replace(' ', '_').replace('/', '_')}.png"
            filepath = os.path.join(LOGO_DOWNLOAD_PATH, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Downloaded logo for {team_name}")
        else:
            print(f"Failed to download logo for {team_name}: {response.status_code}")
    except Exception as e:
        print(f"Error downloading logo for {team_name}: {e}")

def main():
    leagues = get_leagues()
    if not leagues:
        print("No leagues available.")
        return

    # Display leagues
    print("Available Leagues:")
    for idx, league in enumerate(leagues, start=1):
        print(f"{idx}. {league['name']} ({league['area']['name']})")

    # User selects a league
    try:
        choice = int(input("Select a league by number: "))
        if 1 <= choice <= len(leagues):
            selected_league = leagues[choice - 1]
            print(f"Fetching teams for {selected_league['name']}...")
            teams = get_teams(selected_league['id'])
            if teams:
                teams_dict = {}
                for team in teams:
                    team_name = team['name']
                    logo_url = team.get('crest')  # use 'crest' instead of 'crestUrl' in v4
                    teams_dict[team_name] = logo_url
                print("\nTeam Logos Dictionary:")
                for name, url in teams_dict.items():
                    print(f"{name}: {url}")
                return teams_dict
            else:
                print("No teams found for this league.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


if __name__ == "__main__":
    main()


Available Leagues:
1. Premier League (England)
2. Ligue 1 (France)
3. Bundesliga (Germany)
4. Serie A (Italy)
5. Eredivisie (Netherlands)
6. Primeira Liga (Portugal)
7. Primera Division (Spain)
Fetching teams for Premier League...

Team Logos Dictionary:
Arsenal FC: https://crests.football-data.org/57.png
Aston Villa FC: https://crests.football-data.org/58.png
Chelsea FC: https://crests.football-data.org/61.png
Everton FC: https://crests.football-data.org/62.png
Fulham FC: https://crests.football-data.org/63.png
Liverpool FC: https://crests.football-data.org/64.png
Manchester City FC: https://crests.football-data.org/65.png
Manchester United FC: https://crests.football-data.org/66.png
Newcastle United FC: https://crests.football-data.org/67.png
Tottenham Hotspur FC: https://crests.football-data.org/73.png
Wolverhampton Wanderers FC: https://crests.football-data.org/76.png
Leicester City FC: https://crests.football-data.org/338.png
Southampton FC: https://crests.football-data.org/340.pn

In [64]:
def main():
    leagues = get_leagues()
    if not leagues:
        print("No leagues available.")
        return

    # Display leagues
    print("Available Leagues:")
    for idx, league in enumerate(leagues, start=1):
        print(f"{idx}. {league['name']} ({league['area']['name']})")

    # User selects a league
    try:
        choice = int(input("Select a league by number: "))
        if 1 <= choice <= len(leagues):
            selected_league = leagues[choice - 1]
            league_name_clean = selected_league['name'].replace(' ', '_').replace('/', '_')
            print(f"\nFetching teams for {selected_league['name']}...\n")
            teams = get_teams(selected_league['id'])

            if teams:
                teams_dict = {}
                for team in teams:
                    team_name = team['name']
                    logo_url = team.get('crest')  # API v4 uses 'crest'
                    teams_dict[team_name] = logo_url
                    print(f"{team_name}: {logo_url}")

                # Save to text file
                file_path = os.path.join(LOGO_DOWNLOAD_PATH, f"{league_name_clean}.txt")
                with open(file_path, 'w', encoding='utf-8') as f:
                    for name, url in teams_dict.items():
                        f.write(f"{name}: {url}\n")

                print(f"\nSaved teams to {file_path}")
                return teams_dict
            else:
                print("No teams found for this league.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


if __name__ == "__main__":
    main()


Available Leagues:
1. Premier League (England)
2. Ligue 1 (France)
3. Bundesliga (Germany)
4. Serie A (Italy)
5. Eredivisie (Netherlands)
6. Primeira Liga (Portugal)
7. Primera Division (Spain)

Fetching teams for Primera Division...

Athletic Club: https://crests.football-data.org/77.png
Club Atlético de Madrid: https://crests.football-data.org/78.png
CA Osasuna: https://crests.football-data.org/79.png
RCD Espanyol de Barcelona: https://crests.football-data.org/80.png
FC Barcelona: https://crests.football-data.org/81.png
Getafe CF: https://crests.football-data.org/82.png
Real Madrid CF: https://crests.football-data.org/86.png
Rayo Vallecano de Madrid: https://crests.football-data.org/87.png
RCD Mallorca: https://crests.football-data.org/89.png
Real Betis Balompié: https://crests.football-data.org/90.png
Real Sociedad de Fútbol: https://crests.football-data.org/92.png
Villarreal CF: https://crests.football-data.org/94.png
Valencia CF: https://crests.football-data.org/95.png
Real Vallad

In [8]:
import sqlite3

def fetch_all(query, params=()):
    conn = get_connection()
    conn.row_factory = sqlite3.Row  # Enables dict-style row access
    cur = conn.cursor()
    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()
    return [dict(row) for row in rows]

def execute_query(query, params=()):
    conn = get_connection()
    cur = conn.cursor()
    cur.execute(query, params)
    conn.commit()
    conn.close()


In [9]:
def assign_team_to_leagues_console():
    # Fetch all teams
    teams = fetch_all("SELECT id, name FROM teams")
    if not teams:
        print("❌ No teams found.")
        return

    print("\n📋 List of Teams:")
    for i, team in enumerate(teams, start=1):
        print(f"{i}. {team['name']}")

    choice = int(input("\nEnter the number of the team to assign leagues: "))
    selected_team = teams[choice - 1]
    selected_team_id = selected_team['id']

    # Fetch all leagues
    leagues = fetch_all("SELECT id, name FROM leagues")
    if not leagues:
        print("❌ No leagues found.")
        return

    print("\n🏆 Available Leagues:")
    for i, league in enumerate(leagues, start=1):
        print(f"{i}. {league['name']}")

    selected_numbers = input("\nEnter the numbers of the leagues to assign (comma-separated): ")
    selected_indices = [int(x.strip()) for x in selected_numbers.split(",")]
    selected_league_ids = [leagues[i - 1]['id'] for i in selected_indices]

    # Delete old assignments
    execute_query("DELETE FROM team_league WHERE team_id = ?", (selected_team_id,))

    # Insert new assignments
    for league_id in selected_league_ids:
        execute_query(
            "INSERT INTO team_league (team_id, league_id) VALUES (?, ?)",
            (selected_team_id, league_id)
        )

    print(f"\n✅ Successfully updated leagues for team '{selected_team['name']}'.")

assign_team_to_leagues_console()


📋 List of Teams:
1. AFC Bournemouth
2. Al Ahly
3. Al-Ain
4. Al-Hilal
5. Arsenal FC
6. Aston Villa FC
7. Athletic Club
8. Atletico Madrid
9. Barcelona
10. Borussia Dortmund
11. Brentford FC
12. Brighton & Hove Albion FC
13. CA Osasuna
14. Chelsea FC
15. Crystal Palace FC
16. Deportivo Alavés
17. Elche
18. Everton FC
19. FC Bayern München
20. France
21. Fulham FC
22. Germany
23. Getafe CF
24. Girona FC
25. Inter Miami
26. Juventus FC
27. Levante UD
28. Liverpool FC
29. Manchester City FC
30. Manchester United FC
31. Newcastle United FC
32. Nottingham Forest FC
33. Paris Saint-Germain FC
34. Portugal
35. Pyramids
36. RC Celta de Vigo
37. RCD Espanyol
38. RCD Mallorca
39. Rayo Vallecano
40. Real Betis
41. Real Madrid
42. Real Sociedad
43. Sevilla FC
44. Spain
45. Test
46. Tottenham Hotspur FC
47. Valencia CF
48. Villarreal CF
49. West Ham United FC
50. Wolverhampton Wanderers FC
51. Zamalek

🏆 Available Leagues:
1. Bundesliga
2. CAF Champions League
3. Club World Championship
4. Copa del 

In [133]:
table_info("matches")
table_info("rounds")
table_info("two_legged_ties")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score,            │
│ away_score, stage_id, penalty_winner                                                                            │
│ 📊 Total Rows: 6                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from matches                                              
╭────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬─────────╮
│ id │ round_id │ league_… │ home_te… │ away_te… │ match_d… │ status   │ home_sc… │ away_sc… │ stage_id │ penalt… │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 57 │ 25       │ 13       │ 118      │ 121      │ 2025-06… │ finished │ 1        │ 2        │ 222      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 58 │ 25       │ 13       │ 119      │ 120      │ 2025-06… │ finished │ 4        │ 5        │ 222      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 59 │ 26       │ 13       │ 118      │ 120      │ 2025-06… │ finished │ 0        │ 2        │ 223      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 60 │ 26       │ 13       │ 121      │ 119      │ 2025-06… │ finished │ 2        │ 2        │ 224      │ 121     │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 71 │ 26       │ 6        │ 65       │ 83       │ 2025-06… │ finished │ 2        │ 1        │ 220      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 72 │ 26       │ 6        │ 83       │ 65       │ 2025-06… │ finished │ 2        │ 1        │ 220      │ 83      │
╰────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴─────────╯

🔌 Connection closed.

╭────────────────────── Table Info ──────────────────────╮
│ 📋 Table: rounds                                       │
│ 🔢 Columns: id, name, start_date, end_date, created_at │
│ 📊 Total Rows: 2                                       │
╰────────────────────────────────────────────────────────╯

                    Sample Data from rounds                     
╭────┬─────────┬────────────┬────────────┬─────────────────────╮
│ id │ name    │ start_date │ end_date   │ created_at          │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 25 │ Round 1 │ 2025-05-31 │ 2025-06-06 │ 2025-06-10 00:01:30 │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 26 │ Round 2 │ 2025-06-07 │ 2025-06-13 │ 2025-06-10 00:02:24 │
╰────┴─────────┴────────────┴────────────┴─────────────────────╯

🔌 Connection closed.

╭────────────────────────────── Table Info ───────────────────────────────╮
│ 📋 Table: two_legged_ties                                               │
│ 🔢 Columns: id, first_leg_match_id, second_leg_match_id, winner_team_id │
│ 📊 Total Rows: 2                                                        │
╰─────────────────────────────────────────────────────────────────────────╯

                 Sample Data from two_legged_ties                 
╭────┬────────────────────┬─────────────────────┬────────────────╮
│ id │ first_leg_match_id │ second_leg_match_id │ winner_team_id │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 4  │ 69                 │ 70                  │ None           │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 5  │ 71                 │ 72                  │ 83             │
╰────┴────────────────────┴─────────────────────┴────────────────╯

🔌 Connection closed.

In [100]:
def assign_all_matches_to_round(round_id=47):
    query = f"""
    UPDATE matches
    SET round_id = {round_id};
    """
    try:
        execute_query(query)
        print(f"✅ All matches have been assigned to round_id = {round_id}")
    except Exception as e:
        print(f"❌ Error updating matches: {e}")

assign_all_matches_to_round()

📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: FOREIGN KEY constraint failed                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

✅ All matches have been assigned to round_id = 47


In [110]:
def delete_all_records(connection):
    """
    Delete all records from the 'two_legged_ties', 'matches', 'stages', and 'rounds' tables.
    
    Args:
        connection: SQLite3 connection object.
    """
    try:
        cursor = connection.cursor()
        # Disable foreign key constraint temporarily to avoid issues during deletion
        cursor.execute("PRAGMA foreign_keys = OFF;")
        
        # Delete in order from dependent to parent tables
        cursor.execute("DELETE FROM two_legged_ties;")
        cursor.execute("DELETE FROM matches;")
        cursor.execute("DELETE FROM stages;")
        cursor.execute("DELETE FROM rounds;")
        
        connection.commit()
        print("All records deleted successfully.")
    except Exception as e:
        print(f"Error deleting records: {e}")
        connection.rollback()
    finally:
        # Re-enable foreign keys
        cursor.execute("PRAGMA foreign_keys = ON;")
conn = get_connection()
delete_all_records(conn)


All records deleted successfully.


In [35]:
query_delete_predictions = "DELETE FROM predictions;"
execute_query(query_delete_predictions)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [37]:
table_info("players")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: players                                                                                               │
│ 🔢 Columns: id, username, email, password_hash, avatar_path, role, is_confirmed, reset_token, created_at,       │
│ updated_at, last_login_at, avatar_url                                                                           │
│ 📊 Total Rows: 2                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from players                                              
╭────┬─────────┬─────────┬─────────┬─────────┬────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────╮
│ id │ userna… │ email   │ passwo… │ avatar… │ role   │ is_con… │ reset_… │ create… │ update… │ last_l… │ avatar… │
├────┼─────────┼─────────┼─────────┼─────────┼────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 8  │ BaselA… │ BaselA… │ b'$2b$… │ assets… │ admin  │ 1       │ None    │ 2025-0… │ 2025-0… │ 2025-0… │ assets… │
│    │         │         │         │         │        │         │         │ 14:00:… │ 15:24:… │         │         │
├────┼─────────┼─────────┼─────────┼─────────┼────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 14 │ Mohamed │ basela… │ b'$2b$… │ None    │ player │ 1       │ None    │ 2025-0… │ 2025-0… │ 2025-0… │ None    │
│    │ Khaled  │         │         │         │        │         │         │ 18:00:… │ 23:13:… │         │         │
╰────┴─────────┴─────────┴─────────┴─────────┴────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────╯

🔌 Connection closed.

In [28]:
con = get_connection()
con.execute("PRAGMA foreign_keys = ON;")